In [ ]:
import gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt


In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
#param

LEARNING_RATE = 0.005 # pour AdamOptimizer
GAMMA = 0.99 # taux pour reward => advantage

MEMORY_SIZE = 500 # memoire min avant train


In [ ]:
class Agent(tf.keras.Model):

    def __init__(self):
        super(Agent, self).__init__()
        
        # Define your layers here.
        self.dense1 = tf.keras.layers.Dense(24, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, inputs):
        tmp = tf.convert_to_tensor(inputs)
        tmp = self.dense1(tmp)
        tmp = self.dense2(tmp)

        logits = tmp
        logs_pi = tf.math.log(logits)
        return logits, logs_pi

    def explore(self, inputs):
        logits, logs_pi = self(inputs)
        return tf.squeeze(tf.random.categorical(logits, 1)), tf.squeeze(logs_pi), logits

    def exploit(self, inputs):
        logits, _ = self(inputs)
        return tf.math.argmax(logits)


In [ ]:
agent = Agent()
agent.build(tf.TensorShape([None,4]))
#agent.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
#    loss="mse",
#    metrics=['accuracy'])

agent.summary()

In [ ]:
memory = []
nb_steps = []

stat_step = []
stat_loss = []
stat_entrop = []

In [ ]:

class LossFromMemory():
    """ Calcul du loss """
    
    def __init__(self, memory):
        # Define your layers here.
        self.memory = memory
        self.approx_ent = None

    def __call__(self):
        # Récuperation des avantages
        tf_adv = tf.convert_to_tensor( [ mem['ad'] for mem in memory ] )
        
        # Récupérations des états
        states = [ mem['s'] for mem in memory ]
        # Récupération des l'actions choisies
        actions = [ mem['a'] for mem in memory ]
        
        _, logs_pi = agent(states) # Recalcule des logs de la Policy (pour récupérer les gradients)
        logp_a_op = tf.reduce_sum( logs_pi * tf.one_hot(actions, 2), axis=1 ) # Ne conserve que les logs qui corresponds à l'action

        pi_loss = -tf.reduce_mean(logp_a_op * tf_adv) # Calcul l'esperance de (log.pi().advantage)
        
        self.approx_loss = pi_loss
        self.approx_ent = tf.reduce_mean(-logp_a_op) # Calcul de l'entropie
        
        return pi_loss

for epoch in range(1,600) :
    print("Epoch : ", epoch)
    state = env.reset()
    
    step = 0
    epoch_memory = [] # Memoire de l'époque

    while True :
        step += 1

        #env.render()

        action, logs_pi, _sm = agent.explore([state]) # recupere l'action depuis la policy
        if epoch % 20 == 20-1 :
            env.render()
            print(state, action.numpy(), logs_pi.numpy(), _sm.numpy())
        #print(state, action.numpy(), logs_pi.numpy(), _sm.numpy())
        state_next, reward, done, info = env.step(action.numpy()) 
        if done : reward = -1.0 # si finie : reward negatif
        
        epoch_memory.append({ "s": state, "a": action, "r": reward, "l": logs_pi, "d": done, "sn": state_next }) # memorique l'étape
        
        state = state_next

        if done :
            advantage = 0
            for mem in np.flip(epoch_memory) : # Calcule l'avantage
                advantage = advantage * GAMMA + mem['r']
                #mem['ad'] = advantage
                #if mem['ad']>0 : mem['ad'] /= step
                mem['ad'] = advantage / step
                #mem['ad'] = min(advantage, 1)
                #print(advantage)
            
            memory.extend(epoch_memory) # ajout la epoque à la memoire globale
            nb_steps.append(step)
            break
        else :
            print('.', end='')
        
        state = state_next
        
    print(' ', step)
    if len(memory) >= MEMORY_SIZE :
        
        if len(memory)>5000 : memory = np.random.choice(memory, 5000).tolist()
        #pi_loss = LossFromMemory(memory[-5000:])
        pi_loss = LossFromMemory(memory)

        for _ in range(1) :
            train_op = tf.optimizers.Adam(
                    learning_rate=LEARNING_RATE
                ).minimize(
                    loss=pi_loss, 
                    var_list=agent.trainable_variables
                ) # train

            print(np.mean(nb_steps[-100:]), " / loss : ", pi_loss.approx_loss.numpy(), " / entrop : ", pi_loss.approx_ent.numpy())
        stat_step.append(np.mean(nb_steps[-100:]))
        stat_loss.append(pi_loss.approx_loss.numpy())
        stat_entrop.append(pi_loss.approx_ent.numpy())
        
        

In [ ]:
#env.close()
print(len(memory))

plt.figure(figsize=(15.0,10.0))
fig, axes = plt.subplots(nrows=2, ncols=2) 
fig.set_size_inches(15.0, 7.0)         
axes[0][0].plot(stat_step, label="mean steps (last 100)")
axes[0][0].legend()
axes[1][0].plot(stat_loss, label="loss")
axes[1][0].legend()
axes[0][1].plot(stat_entrop, label="entropy")
axes[0][1].legend()
plt.show()


In [ ]:
from matplotlib import interactive
interactive(True)

In [ ]:
#agent.get_weights()

state = env.reset()

step = 0
while True :
    step += 1

    env.render()

    #action = agent.exploit([state])
    logits, _ = agent.call([state])
    logits = tf.squeeze(logits)
    action = tf.math.argmax(logits) #, axis=1)
    print(step, state, action.numpy(), logits.numpy())
    state, _, done, _ = env.step(action.numpy()) 
    if done : break
